In [1]:
import datetime, time, math, os, json, shutil, asyncio, networkx as nx
from utils.gitutils import create_pull_request, clone_repo, create_branch, wipe_repo, prepare_repo
from utils.agent import Agent, GenerationConfig, Interaction, Team
from utils.stringutils import arr_from_sep_string, extract_markdown_blocks, markdown_to_dict, fuzzy_find, skwonk
from utils.filetreeutils import FileTree, write_file_tree
from utils.listutils import flatten
from utils.frameworkutils import DartAnalyzer
from dotenv import load_dotenv
from utils.graphutils import loose_level_order, collapsed_level_order
from utils.frameworkutils import Framework

In [2]:
load_dotenv()  # Load environment variables from .env

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
API_KEY = os.getenv('API_KEY')

In [3]:
repo_url = 'https://github.com/6165-MSET-CuttleFish/TeamTrack'
source = Framework.FLUTTER
feature = f"""## Feature Name
Spaces
## Descriptions
Organizations are a way for users to share videos of their robot competing. These videos will have information about the team number and name as well. Users will be able to view these videos and see how other teams are doing. They will also be able to upload their own videos to share with others.
"""

In [4]:
# Clone the repo and make a new branch
repo = clone_repo(repo_url)
base_branch = repo.active_branch.name
created_branch = f"modification-{base_branch}"
local_repo_path = str(repo.working_dir)
working_dir_path = f'{local_repo_path}\\{source.get_working_dir()}'

In [5]:
source_swe_summarizer = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_analyzer",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to summarize the functionality of provided code files. Structure your response as follows:

# filename.ext:

$7 sentence summary of the functionality/contents of the file$
'''
)
pm = Agent(
    model_name="gemini-1.5-pro-001",
    api_key=API_KEY,
    name="pm",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a high-level technical project manager tasked with the project of adding functionality to a {source} repository."""
)
source_swe_coder = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_coder",
    generation_config=GenerationConfig(temperature=0.5),
    system_prompt=f"You are a {source} software engineer. Your job is to write code to implement a specific feature. Respond with code and nothing else. No explanations needed."
)
source_swe_editor = Agent(
    model_name="gemini-1.5-flash",
    api_key=API_KEY,
    name="source_swe_editor",
    generation_config=GenerationConfig(temperature=0.0),
    system_prompt=f"""You are a {source} software engineer tasked with editing a large code file to implement changes in accordance with the instructions you will receive in the following prompts. Large edits at once are NOT the way. We will solve this problem by dividing the edit into smaller sub-edits consisting of additions and deletions (no more than 10 lines per edit). We will use a custom file type called 'skinnydiff', which are diff-like files but only feature a larger file's substring, to convey these changes. Here is an example of a 'skinnydiff' file:

```skinnydiff
 void foo() {{
- int x = 5;
+ var x = 7;
  return x + 3; // Example code 
 }}
```
Skinnydiffs are shorter versions of diff files that only include a few context lines around the deletions and insertions. They DO NOT CONTAINS @@...@@  as this is wasted information

Given this knowledge, structure your response as follows:

# Step 1
$Talk through the 1st sub-change and what you plan to do$
## Scope
$Brief planning on which line or class to include in the change$
## Change 1
```skinnydiff
$code diff$
```

# Step N
$Talk through the Nth sub-change and what you plan to do$
## Scope
$Brief planning on which line or class to include in the change$
## Change N
```skinnydiff
$...$
```"""
)
skeletonizer = Agent(
    name="skeletonizer",
    model_name="gemini-1.5-flash",
    api_key=API_KEY,
    system_prompt="""Why say many word when few word do trick?! Turn code into skeleton-pseudocode by condensing nitty gritty syntax into sensible short pseudocode. Respond as such: 
```pseudocode
....
```""",
    generation_config=GenerationConfig(temperature=0.0)
)

team = Team(source_swe_summarizer, pm, source_swe_coder, source_swe_editor, skeletonizer)

In [6]:
analyzer = source.get_analyzer(working_dir_path)

source_dependency_graph = analyzer.buildDependencyGraph()

In [7]:
# loose level order
eval_order = loose_level_order(source_dependency_graph)[::-1]
for level in eval_order:
    print(level)
print(f"Number of levels: {len(eval_order)}")

['models\\AppModel.dart', 'models\\StatConfig.dart']
['models\\GameModel.dart']
['models\\ScoreModel.dart']
['functions\\Extensions.dart', 'components\\misc\\PlatformGraphics.dart']
['components\\statistics\\BarGraph.dart', 'components\\users\\PFP.dart']
['components\\scores\\ScoreSummary.dart', 'components\\scores\\Incrementor.dart', 'providers\\Auth.dart', 'components\\users\\UsersRow.dart', 'functions\\Statistics.dart']
['views\\home\\match\\MatchView.dart']
['components\\statistics\\PercentChange.dart', 'functions\\Functions.dart']
['components\\scores\\ScoreTimeline.dart', 'components\\scores\\ScoreRangeSummary.dart', 'models\\Change.dart']
['views\\home\\change\\ChangeConfig.dart', 'components\\statistics\\CheckList.dart', 'views\\home\\match\\MatchConfig.dart', 'components\\misc\\CardView.dart', 'components\\scores\\ScoringElementStats.dart', 'components\\misc\\EmptyList.dart', 'views\\home\\change\\ChangeRow.dart', 'views\\home\\match\\ExampleMatchRow.dart', 'views\\home\\match

In [31]:
(ranks := sorted(nx.pagerank(source_dependency_graph).items(), key=lambda x: x[1], reverse=True))

[('functions\\Functions.dart', 0.13015165084511265),
 ('models\\AppModel.dart', 0.1133542274603946),
 ('models\\GameModel.dart', 0.11054238484593704),
 ('components\\misc\\PlatformGraphics.dart', 0.07280617686501588),
 ('functions\\Extensions.dart', 0.06491036089133999),
 ('models\\ScoreModel.dart', 0.04846332476566593),
 ('functions\\Statistics.dart', 0.04758879390256111),
 ('models\\StatConfig.dart', 0.03794723616207956),
 ('views\\home\\match\\MatchView.dart', 0.03251008281339123),
 ('models\\Change.dart', 0.02251042215978528),
 ('providers\\Auth.dart', 0.01548168224778635),
 ('components\\statistics\\BarGraph.dart', 0.012206693129943554),
 ('components\\users\\PFP.dart', 0.009606083157777829),
 ('models\\GPTModel.dart', 0.009186070453302648),
 ('views\\LandingPage.dart', 0.008410495014922857),
 ('components\\scores\\ScoreSummary.dart', 0.007612721724091385),
 ('components\\users\\UsersRow.dart', 0.007612721724091385),
 ('components\\misc\\EmptyList.dart', 0.0075104656743458665),
 (

In [35]:
max_probability = ranks[0][1]
(percentage_of_max_arr := [(rank[0], rank[1] / max_probability) for rank in ranks])

[('functions\\Functions.dart', 1.0),
 ('models\\AppModel.dart', 0.8709396056396712),
 ('models\\GameModel.dart', 0.8493352495197185),
 ('components\\misc\\PlatformGraphics.dart', 0.5593949549795498),
 ('functions\\Extensions.dart', 0.4987286789668673),
 ('models\\ScoreModel.dart', 0.3723604307050999),
 ('functions\\Statistics.dart', 0.3656411086110179),
 ('models\\StatConfig.dart', 0.29156169680274574),
 ('views\\home\\match\\MatchView.dart', 0.24978617330086691),
 ('models\\Change.dart', 0.17295533336395305),
 ('providers\\Auth.dart', 0.11895110163612424),
 ('components\\statistics\\BarGraph.dart', 0.09378823127238059),
 ('components\\users\\PFP.dart', 0.07380684836037597),
 ('models\\GPTModel.dart', 0.0705797459629195),
 ('views\\LandingPage.dart', 0.06462073250943079),
 ('components\\scores\\ScoreSummary.dart', 0.05849116530339616),
 ('components\\users\\UsersRow.dart', 0.05849116530339616),
 ('components\\misc\\EmptyList.dart', 0.05770549682296168),
 ('components\\scores\\Increment

In [8]:
(source_file_tree := FileTree.from_dir(working_dir_path))

├── api\
│   ├── README.MD
├── components\
│   ├── misc\
│   │   ├── CardView.dart
│   │   ├── Collapsible.dart
│   │   ├── EmptyList.dart
│   │   ├── InfoPills.dart
│   │   ├── PlatformGraphics.dart
│   ├── scores\
│   │   ├── Incrementor.dart
│   │   ├── ScoreCard.dart
│   │   ├── ScoreRangeSummary.dart
│   │   ├── ScoreSummary.dart
│   │   ├── ScoreTimeline.dart
│   │   ├── ScoringElementStats.dart
│   ├── statistics\
│   │   ├── BarGraph.dart
│   │   ├── CheckList.dart
│   │   ├── PercentChange.dart
│   ├── users\
│   │   ├── PFP.dart
│   │   ├── UsersRow.dart
│   ├── AutonomousDrawingTool.dart
├── functions\
│   ├── APIMethods.dart
│   ├── Extensions.dart
│   ├── Functions.dart
│   ├── GPTModel.dart
│   ├── ResponseModel.dart
│   ├── Statistics.dart
├── models\
│   ├── AppModel.dart
│   ├── Change.dart
│   ├── GameModel.dart
│   ├── GPTModel.dart
│   ├── ScoreModel.dart
│   ├── StatConfig.dart
├── providers\
│   ├── Auth.dart
│   ├── PushNotifications.dart
│   ├── Theme.dart
│   ├

In [9]:
async def skeletonize(node):
    name = source_file_tree.nodes[node]['name']
    content = source_file_tree.nodes[node]['content']
    message = f"{name}\n```\n{content}\n```"
    return await team.async_chat_with_agent(
        agent_name='skeletonizer',
        message=message,
        save_keys=[f"skeleton_{node}", "all_skeletons"],
        prompt_title=f"Skeleton of {node}"
    )
order = source_file_tree.get_files()
tasks = [skeletonize(node) for node in order]
skeletons = await asyncio.gather(*tasks)
for i, response in enumerate(skeletons):
    source_file_tree.nodes[order[i]]['skeleton'] = response

In [10]:
# Summarize the original repo
async def summarize(node):
    name = source_file_tree.nodes[node]['name']
    content = source_file_tree.nodes[node]['content']
    message = f"{name}\n```\n{content}\n```"
    return await team.async_chat_with_agent(
        agent_name='source_swe_analyzer',
        message=message,
        context_keys=[f"skeleton_{neighbor}" for neighbor in source_dependency_graph[node]],
        save_keys=[f"summary_{node}", "all_summaries"],
        prompt_title=f"Summary of {node}"
    )
tasks = [summarize(node) for node in order]
responses = await asyncio.gather(*tasks)
for i, response in enumerate(responses):
    source_file_tree.nodes[order[i]]["summary"] = response

In [11]:
prompt = f"""This is the file tree for the original {source} repo:
```
{source.get_working_dir()}\\
{source_file_tree}
```
Given the prior context, summarize the functionality of the entire repository succinctly.
"""
print(team.chat_with_agent('source_swe_analyzer', prompt, context_keys=["all_summaries"], save_keys=["all_summaries"], prompt_title=f"""Repository Summary.
File Tree provided:
```
{source.get_working_dir()}\\
{source_file_tree}
```
"""))

This Flutter repository implements a comprehensive application for managing and analyzing data related to FIRST Robotics Competition (FRC) events. It provides features for creating and managing events, teams, and matches, as well as visualizing team performance and simulating alliance formations. The application leverages Firebase for data persistence and real-time updates, and utilizes Google's ML Kit for image analysis. It includes features for:

* **Event Management:** Creating, sharing, and managing events, including adding teams and matches.
* **Team Management:** Viewing team details, scores, match history, and autonomous paths.
* **Match Management:** Creating, viewing, and editing match details, including scores and penalties.
* **Data Analysis:** Displaying statistical data, including score summaries, bar graphs, and line charts.
* **Alliance Formation:** Simulating alliance formations, generating recommendations based on team performance, and viewing alliance details.
* **Use

## Summary Compression

In [12]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all_summaries'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

16959

In [13]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [14]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.86206700230173

## Skeleton Compression

In [15]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all_skeletons'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

35555

In [16]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [17]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.7108197574643558

In [18]:
response_format = f"""
You may choose ACTIONS from the following list:
- | WRITE | $FILE_PATH$ | $COMMA_SEPERATED_DEPENDENCY_PATH(S)$ | $PROMPT$ (PROMPT should be a brief instruction for how the new FILE_PATH should be WRITTEN)
- | EDIT | $FILE_PATH$ | $PROMPT$ (PROMPT should be a brief instruction for how the FILE_PATH should be EDITED)
- | DELETE | $FILE_PATH$ (Delete the provided file)
----------
Structure your response using the following example:
```
| WRITE | {source.get_working_dir()}\\path\\to\\newfile.ext | | Implement ... with the following properties:
    - property1
    - property2
    - dependency1
    - dependency2
| EDIT | {source.get_working_dir()}\\path\\to\\file.ext | Fix the bug ... by doing:
    - step1
    - step2
| DELETE | {source.get_working_dir()}\\path\\to\\some\\file.ext | Short deletion message | Internal reasons for deletion:
    - reason1
    - reason2
```
It is VERY important that you only respond with a high level overview of what to do. The actual implementation will be done by the coder.
"""

# Create new file tree
prompt = f'''
You are to build the following feature:
{feature}
------------------
{response_format}
------------------
You need to follow the response template structure exactly (MY JOB DEPENDS ON IT)
'''
print(response := team.chat_with_agent("pm", prompt, context_keys=["all_skeletons"]))

```
| WRITE | models\Space.dart | Implement the Space model with the following properties:
    - id: string (unique identifier for the space)
    - name: string (name of the organization)
    - members: list of user IDs (users who are members of the space)
    - videos: list of video IDs (videos shared within the space)
| WRITE | models\Video.dart | Implement the Video model with the following properties:
    - id: string (unique identifier for the video)
    - spaceId: string (ID of the space the video belongs to)
    - userId: string (ID of the user who uploaded the video)
    - url: string (URL of the video)
    - teamNumber: int (team number featured in the video)
    - teamName: string (team name featured in the video)
    - timestamp: DateTime (time the video was uploaded)
| WRITE | views\spaces\SpaceList.dart | Implement a screen to display a list of spaces the user is a member of. Allow users to create new spaces.
| WRITE | views\spaces\SpaceView.dart | Implement a screen to di

In [19]:
md = extract_markdown_blocks(response)[0]
blocks = md.split("|")
actions = []
i = 0
while i < len(blocks):
    block = blocks[i].strip()
    if block != "WRITE" and block != "EDIT" and block != "DELETE":
        i += 1
        continue
    action, file, prompt = blocks[i:i + 3]
    action = action.strip()
    if file.strip().startswith(source.get_working_dir()):
        file = file.strip()[len(source.get_working_dir()) + 1:]
    else:
        file = file.strip()
    prompt = prompt.strip()
    actions.append((action, file, prompt))
    i += 3
for action in actions:
    print(action)

('WRITE', 'models\\Space.dart', 'Implement the Space model with the following properties:\n    - id: string (unique identifier for the space)\n    - name: string (name of the organization)\n    - members: list of user IDs (users who are members of the space)\n    - videos: list of video IDs (videos shared within the space)')
('WRITE', 'models\\Video.dart', 'Implement the Video model with the following properties:\n    - id: string (unique identifier for the video)\n    - spaceId: string (ID of the space the video belongs to)\n    - userId: string (ID of the user who uploaded the video)\n    - url: string (URL of the video)\n    - teamNumber: int (team number featured in the video)\n    - teamName: string (team name featured in the video)\n    - timestamp: DateTime (time the video was uploaded)')
('WRITE', 'views\\spaces\\SpaceList.dart', 'Implement a screen to display a list of spaces the user is a member of. Allow users to create new spaces.')
('WRITE', 'views\\spaces\\SpaceView.dart'

In [22]:
async def write(file_path, prompt):
    response = await source_swe_coder.async_chat(f"Instructions for {file_path}:\n{prompt}")
    md = extract_markdown_blocks(response)[0]
    path = f"{working_dir_path}\\{file_path}"
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, 'w') as f:
        f.write(md)

async def edit(file_path, prompt):
    with open(f"{working_dir_path}\\{file_path}", 'r') as f:
        content = f.read()
    response = await source_swe_editor.async_chat(
        f"{prompt}\n\nRemember, keep your skinnydiffs SHORT and do NOT include @@...@@. Instead, split it into multiple steps. Anytime you see @@...@@, just split it into another step.",
        custom_context=[Interaction(f"Contents of {file_path}", content)]
    )
    diffs = extract_markdown_blocks(response)
    for diff in diffs:
        with open(f'{working_dir_path}\\{file_path}', 'w') as f:
            f.seek(0)
            f.truncate()
            content = skwonk(content, diff)
            f.write(content)

async def delete(file_path):
    os.remove(f"{working_dir_path}\\{file_path}")

In [23]:
from utils.errorutils import repeat_until_finish
from tqdm import tqdm

writes = [repeat_until_finish(lambda: write(file, prompt)) for action, file, prompt in actions if action == 'WRITE']
edits = [repeat_until_finish(lambda: edit(file, prompt)) for action, file, prompt in actions if action == 'EDIT']
deletes = [repeat_until_finish(lambda: delete(file)) for action, file, prompt in actions if action == 'DELETE']

await asyncio.gather(*writes, *edits, *deletes)

 56%|█████▌    | 5/9 [00:11<00:10,  2.62s/it]

index: 0
finish_reason: RECITATION



100%|██████████| 9/9 [00:34<00:00,  3.81s/it]
